In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
############################################## ACTUALIZAR CARTERA CCD ##############################################

In [3]:
columnas = ['DEUDOR', 'NOMBRE', 'RUC', 'ANALISTA_ACT', 'TIPO_DAC', 'ESTADO']
no_analistas = ['REGION NORTE', 'REGION SUR', 'WALTER LOPEZ', 'SIN INFORMACION']
no_estados = ['LIQUIDADO', 'PROCESO DE LIQUIDACIÓN']

# Depurar Nuevo_DACxANALISTA
df_dacx = pd.read_excel('./Nuevo_DACxANALISTA.xlsx', sheet_name='Base_NUEVA', usecols=columnas)
df_dacx = df_dacx.rename(columns={'ANALISTA_ACT': 'ANALISTA', 'TIPO_DAC': 'TIPO'})
df_dacx = df_dacx[~df_dacx['ANALISTA'].isin(no_analistas)]
df_dacx = df_dacx[~df_dacx['ESTADO'].isin(no_estados)]
df_dacx['DEUDOR'] = df_dacx['DEUDOR'].astype("Int64").astype(str)
df_dacx['RUC'] = df_dacx['RUC'].astype("Int64").astype(str)
df_dacx.dropna(subset=['DEUDOR'], inplace=True)
df_dacx.drop_duplicates(subset=['DEUDOR'], inplace=True)
df_dacx.reset_index(drop=True, inplace=True)

list_analistas = df_dacx['ANALISTA'].unique() # Lista de analistas

# Actualizar CARTERA_GENERAL_CCD
df_cartera = pd.read_excel('./CARTERA_CCD.xlsx', sheet_name='GENERAL', usecols=['DEUDOR', 'APOYO1', 'APOYO2', 'APOYO3'])
df_cartera['DEUDOR'] = df_cartera['DEUDOR'].astype("Int64").astype(str)
columnas_cruce = ['DEUDOR', 'NOMBRE', 'ANALISTA', 'APOYO1', 'APOYO2', 'APOYO3', 'ESTADO', 'TIPO']
df_cartera = pd.merge(df_cartera, df_dacx, on='DEUDOR', how='right')
df_cartera = df_cartera[columnas_cruce]
df_cartera.dropna(subset=['DEUDOR'], inplace=True)
df_cartera.drop_duplicates(subset=['DEUDOR'], inplace=True)
df_cartera.sort_values(by='DEUDOR', inplace=True, ignore_index=True)
df_cartera.reset_index(drop=True, inplace=True)
df_cartera.to_excel('./CARTERA_CCD.xlsx', sheet_name='GENERAL', index=False)

# Crear hojas por analista
with pd.ExcelWriter('./CARTERA_CCD.xlsx') as writer:
    df_temp = df_cartera.copy()
    df_cartera.to_excel(writer, sheet_name='GENERAL', index=False)
    for analista in list_analistas:
        df_cartera = df_temp[df_temp['ANALISTA'] == analista]
        df_cartera.sort_values(by='DEUDOR', inplace=True, ignore_index=True)
        df_cartera.reset_index(drop=True, inplace=True)
        print(f'HOJA: {analista} [{df_cartera.shape[0]}]')
        df_cartera.to_excel(writer, sheet_name=analista, index=False)

HOJA: DIEGO RODRIGUEZ [85]
HOJA: JOSE LUIS VALVERDE [83]
HOJA: JUAN CARLOS HUATAY [76]
HOJA: YOLANDA OLIVA [78]
HOJA: RAQUEL CAYETANO [81]


In [4]:
############################################## CRUCE BASE CELULARES vs CARTERA CCD ##############################################

In [5]:
df_celulares = pd.read_excel('./Base_Celulares_CCD.xlsx')
df_celulares['DEUDOR'] = df_celulares['DEUDOR'].astype("Int64").astype(str)
df_celulares['CELULAR'] = df_celulares['CELULAR'].astype("Int64").astype(str)
df_celulares = df_celulares[['DEUDOR', 'CELULAR']]
df_celulares['CELULAR'] = df_celulares[df_celulares['CELULAR'].str.len() == 9]['CELULAR']
df_celulares.dropna(subset=['CELULAR'], inplace=True)
df_celulares['CELULAR'] = '51' + df_celulares['CELULAR']
df_celulares.reset_index(drop=True, inplace=True)

df_cartera = pd.read_excel('./CARTERA_CCD.xlsx', sheet_name='GENERAL', usecols=['DEUDOR', 'ANALISTA', 'APOYO1', 'APOYO2', 'APOYO3'])
df_cartera['DEUDOR'] = df_cartera['DEUDOR'].astype("Int64").astype(str)
df_cruce = pd.merge(df_celulares, df_cartera, on='DEUDOR', how='left')
df_cruce.dropna(subset=['ANALISTA'], inplace=True)
df_cruce.dropna(subset=['CELULAR'], inplace=True)
df_cruce.reset_index(drop=True, inplace=True)
df_cruce.head(5)

,DEUDOR,CELULAR,ANALISTA,APOYO1,APOYO2,APOYO3
0,4070709,51997108820,YOLANDA OLIVA,DIEGO RODRIGUEZ,JOSE LUIS VALVERDE,JUAN CARLOS HUATAY
1,4071078,51997108224,JOSE LUIS VALVERDE,DIEGO RODRIGUEZ,YOLANDA OLIVA,RAQUEL CAYETANO
2,4070112,51980588839,JUAN CARLOS HUATAY,JOSE LUIS VALVERDE,RAQUEL CAYETANO,YOLANDA OLIVA
3,4070230,51950304684,YOLANDA OLIVA,RAQUEL CAYETANO,DIEGO RODRIGUEZ,JOSE LUIS VALVERDE
4,4070266,51979717800,YOLANDA OLIVA,JOSE LUIS VALVERDE,RAQUEL CAYETANO,DIEGO RODRIGUEZ


In [6]:
############################################## CADENA DE TEXTO ##############################################

In [7]:
df_regla = pd.read_excel('./TEXTO.xlsx', sheet_name='TEXTO')
texto1 = df_regla['TEXTO_1'][0]
texto2 = df_regla['TEXTO_2'][0]
texto3 = df_regla['TEXTO_3'][0]
texto4 = df_regla['TEXTO_4'][0]
print(f'{texto1}\n{texto2}\n{texto3}\n{texto4}')

# (f'{row["CELULAR"]}{texto1}{row["ANALISTA"]}{texto2}{row["FECHA_RETORNO"]}{texto3}{row["APOYO"]}{texto4}')

,Text,"Estimado Socio: Le informamos que su analista 
, se encontra de vacaciones hasta el dia 
. Quien quedara a cargo de la cuenta comercial sera el/la analista 
. Gracias por su comprension. Creditos y Cobranzas Distribuidores le desea un excelente dia"


In [8]:
############################################## ANALISTAS DE VACACIONES ##############################################

In [9]:
from datetime import datetime

fecha_hoy = datetime.now().strftime('%d.%m.%Y')

df_vacaciones = pd.read_excel('./VACACIONES.xlsx')
df_vacaciones.dropna(inplace=True)
dict_analistas = dict(zip(df_vacaciones['ANALISTA'], zip(df_vacaciones['FECHA_SALIDA'], df_vacaciones['FECHA_RETORNO'])))

vacaciones = {}
for analista, fechas in dict_analistas.items():
    if fechas[0] <= fecha_hoy <= fechas[1]:
        vacaciones.update({analista: fechas[1]})
vacaciones

{'JUAN CARLOS HUATAY': '25.05.2024', 'RAQUEL CAYETANO': '30.05.2024'}

In [10]:
############################################## FILTRO VACACIONES ##############################################

In [11]:
analistas = list(vacaciones.keys())
columnas_cruce = ['CELULAR', 'ANALISTA', 'FECHA_RETORNO','APOYO']
df_cruce = df_cruce[df_cruce['ANALISTA'].isin(analistas)]
df_cruce['FECHA_RETORNO'] = df_cruce['ANALISTA'].map(vacaciones)
df_cruce['APOYO'] = df_cruce[['APOYO1', 'APOYO2', 'APOYO3']].apply(lambda x: x[0] if x[0] not in (analistas) else (x[1] if x[1] not in (analistas) else x[2]), axis=1)
df_cruce.reset_index(drop=True, inplace=True)
df_cruce = df_cruce[columnas_cruce]
df_cruce.head(5)

,CELULAR,ANALISTA,FECHA_RETORNO,APOYO
0,51980588839,JUAN CARLOS HUATAY,25.05.2024,JOSE LUIS VALVERDE
1,51966773333,JUAN CARLOS HUATAY,25.05.2024,JOSE LUIS VALVERDE
2,51974634230,RAQUEL CAYETANO,30.05.2024,YOLANDA OLIVA
3,51940390901,RAQUEL CAYETANO,30.05.2024,JOSE LUIS VALVERDE
4,51964237590,RAQUEL CAYETANO,30.05.2024,JOSE LUIS VALVERDE


In [12]:
# (f'{row["CELULAR"]}{texto1}{row["ANALISTA"]}{texto2}{row["FECHA_RETORNO"]}{texto3}{row["APOYO"]}{texto4}')
df_cruce['TEXTO'] = df_cruce.apply(lambda row: f'{row["CELULAR"]}{texto1}{row["ANALISTA"]}{texto2}{row["FECHA_RETORNO"]}{texto3}{row["APOYO"]}{texto4}', axis=1)
df_cruce.head(5)

,CELULAR,ANALISTA,FECHA_RETORNO,APOYO,TEXTO
0,51980588839,JUAN CARLOS HUATAY,25.05.2024,JOSE LUIS VALVERDE,"51980588839,Text,""Estimado Socio: Le informamo..."
1,51966773333,JUAN CARLOS HUATAY,25.05.2024,JOSE LUIS VALVERDE,"51966773333,Text,""Estimado Socio: Le informamo..."
2,51974634230,RAQUEL CAYETANO,30.05.2024,YOLANDA OLIVA,"51974634230,Text,""Estimado Socio: Le informamo..."
3,51940390901,RAQUEL CAYETANO,30.05.2024,JOSE LUIS VALVERDE,"51940390901,Text,""Estimado Socio: Le informamo..."
4,51964237590,RAQUEL CAYETANO,30.05.2024,JOSE LUIS VALVERDE,"51964237590,Text,""Estimado Socio: Le informamo..."


In [13]:
sms_vacaciones = df_cruce["TEXTO"].to_list()
with open('SMS_APOYOS.txt', "w") as f:
    for item in sms_vacaciones:
        f.write("%s\n" % item)

import os
os.startfile('SMS_APOYOS.txt')